In [22]:
# pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
# pip install -qU "langchain[aws]"
# pip install python-dotenv
# pip install -qU langchain-aws
# pip install -qU langchain-core

# Investigate:
# pip install unstructured
# pip install "unstructured[pdf]"

### Set Up Credentials and Envitoment Vars

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_REGION'] = os.getenv('AWS_REGION')




### Initilaize Gen LLM

In [24]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

### Initialize Embed LLM

In [25]:
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")


### In Memory Vector Store

In [26]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)



### Loading PDF Documents from a Directory :)

In [27]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Path to your data directory
data_directory = "data"
docs = []

print(f"Loading documents from: {data_directory}")

# Loop through each file in the directory
for filename in os.listdir(data_directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(data_directory, filename)
        print(f"--> Loading {filename}...")
        try:
            # Use PyPDFLoader for each file
            loader = PyPDFLoader(file_path)
            # Extend the main docs list with the pages from the current PDF
            docs.extend(loader.load())
            print(f"    ...success!")
        except Exception as e:
            # If a file fails to load, print an error and continue
            print(f"    ...ERROR loading {filename}: {e}")

print(f"\nSuccessfully loaded a total of {len(docs)} pages from all documents.")

Loading documents from: data
--> Loading tesla 10q.pdf...
    ...success!
--> Loading nvidia 10q.pdf...
    ...success!
--> Loading May Apple 10q.pdf...
    ...success!

Successfully loaded a total of 111 pages from all documents.


### Splitting documents via character

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)
print(f"Split all documents into {len(all_splits)} sub-documents.")


Split all documents into 469 sub-documents.


### Embedding in a vector store

In [29]:
document_ids = vector_store.add_documents(documents=all_splits)


Error raised by inference endpoint
Traceback (most recent call last):
  File "/Users/ktejwani/Local_Ollama_RAG/venv/lib/python3.13/site-packages/urllib3/connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
    ~~~~~~~~~~~~~~~~~~~^^^^^^
  File "/Users/ktejwani/Local_Ollama_RAG/venv/lib/python3.13/site-packages/urllib3/connectionpool.py", line 1093, in _validate_conn
    conn.connect()
    ~~~~~~~~~~~~^^
  File "/Users/ktejwani/Local_Ollama_RAG/venv/lib/python3.13/site-packages/urllib3/connection.py", line 790, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
        sock=sock,
    ...<14 lines>...
        assert_fingerprint=self.assert_fingerprint,
    )
  File "/Users/ktejwani/Local_Ollama_RAG/venv/lib/python3.13/site-packages/urllib3/connection.py", line 969, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
        sock=sock,
    ...<8 lines>...
        tls_in_tls=tls_in_tls,
    )
  File "/Users/ktejwani/Local_Ollam

SSLError: SSL validation failed for https://bedrock-runtime.us-west-2.amazonaws.com/model/amazon.titan-embed-text-v2%3A0/invoke [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)